In [1]:
# stdlib
from functools import partial
from importlib import reload
import sys
from typing import List
from typing import Optional
from typing import Tuple
from typing import Union as TypeUnion
import syft as sy
# third party
import pytest

# syft absolute
import syft
from syft.ast import add_dynamic_objects
from syft.ast.globals import Globals
from syft.core.node.abstract.node import AbstractNodeClient
from syft.core.node.common.client import Client
from syft.lib import lib_ast

import module_test

sys.modules["module_test"] = module_test

module_test_methods = [
    ("module_test.A", "module_test.A"),
    ("module_test.A.__len__", "syft.lib.python.Int"),
    ("module_test.A.__iter__", "syft.lib.python.Iterator"),
    ("module_test.A.__next__", "syft.lib.python.Int"),
    ("module_test.A.test_method", "syft.lib.python.Int"),
    ("module_test.A.test_property", "syft.lib.python.Float"),
    ("module_test.A._private_attr", "syft.lib.python.Float"),
    ("module_test.A.static_method", "syft.lib.python.Float"),
    ("module_test.A.static_attr", "syft.lib.python.Int"),
    ("module_test.B.Car", "module_test.B"),
    ("module_test.C", "module_test.C"),
    ("module_test.C.type_reload_func", "syft.lib.python._SyNone"),
    ("module_test.C.obj_reload_func", "syft.lib.python._SyNone"),
    ("module_test.C.dummy_reloadable_func", "syft.lib.python.Int"),
    ("module_test.global_value", "syft.lib.python.Int"),
    ("module_test.global_function", "syft.lib.python.Int"),
]

dynamic_objects = [("module_test.C.dynamic_object", "syft.lib.python.Int")]


def update_ast_test(
    ast_or_client: TypeUnion[Globals, AbstractNodeClient],
    methods: List[Tuple[str, str]],
    dynamic_objects: Optional[List[Tuple[str, str]]] = None,
) -> None:
    """Checks functionality of update_ast, uses create_ast"""
    if isinstance(ast_or_client, Globals):
        ast = ast_or_client
        test_ast = create_ast_test(
            client=None, methods=methods, dynamic_objects=dynamic_objects
        )
        ast.add_attr(attr_name="module_test", attr=test_ast.attrs["module_test"])
    elif isinstance(ast_or_client, AbstractNodeClient):
        client = ast_or_client
        test_ast = create_ast_test(
            client=client, methods=methods, dynamic_objects=dynamic_objects
        )
        client.lib_ast.attrs["module_test"] = test_ast.attrs["module_test"]
        setattr(client, "module_test", test_ast.attrs["module_test"])
    else:
        raise ValueError(
            f"Expected param of type (Globals, AbstractNodeClient), but got {type(ast_or_client)}"
        )


def create_ast_test(
    client: Optional[AbstractNodeClient],
    methods: List[Tuple[str, str]],
    dynamic_objects: Optional[List[Tuple[str, str]]],
) -> Globals:
    """Unit test for create_ast functionality"""
    ast = Globals(client)

    for method, return_type in methods:
        ast.add_path(
            path=method, framework_reference=module_test, return_type_name=return_type
        )

    if dynamic_objects:
        add_dynamic_objects(ast, dynamic_objects)

    for klass in ast.classes:
        klass.create_pointer_class()
        klass.create_send_method()
        klass.create_storable_object_attr_convenience_methods()

    return ast


def register_module_test() -> None:
    """Test which is required for every other tests (runs first even in random execution)"""
    # Make lib_ast contain the specific methods/attributes
    update_ast_test(
        ast_or_client=syft.lib_ast,
        methods=module_test_methods,
        dynamic_objects=dynamic_objects,
    )

    # Make sure that when we register a new client it would update the specific AST
    lib_ast.loaded_lib_constructors["module_test"] = partial(
        update_ast_test, methods=module_test_methods, dynamic_objects=dynamic_objects
    )



def custom_client() -> Client:
    """Return VM for unit tests"""
    alice = syft.VirtualMachine(name="alice")
    alice_client = alice.get_root_client()

    return alice_client

register_module_test()

root_client = syft.Domain("me").get_root_client()

a = module_test.A()

INITIALIZING IN MEMORY STORE!!!
INITIALIZING IN MEMORY STORE!!!


In [2]:
blob = sy.serialize(a, to_bytes=True)

In [3]:
obj = sy.deserialize(blob, from_bytes=True)

ReferenceError: Reference don't match: module_test

In [6]:
value_to_set = 7.5

a_ptr = root_client.module_test.A()
# a_ptr.test_property = value_to_set
# result_ptr = a_ptr.test_property

# a = module_test.A()
# a.test_property = value_to_set
# result = a.test_property


# assert result == result_ptr.get()

[2021-06-26T21:25:45.996471+0100][CRITICAL][logger]][61657] Object <class 'module_test.A'> is not serializable and has no _sy_serializable_wrapper_type


Storing object:<Storable: <module_test.A object at 0x7fd9d0ffc670>>


Exception: Object <class 'module_test.A'> is not serializable and has no _sy_serializable_wrapper_type